## Importação de Bibliotecas e Preparação dos Dados

Configura o ambiente, carrega os dados de transações de cartão de crédito e prepara as variáveis preditoras e a variável alvo.

In [20]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import numpy as np

# Carregar os dados
file_path = './creditcard.csv'  # Caminho para o arquivo
df = pd.read_csv(file_path)

# Preparar os dados
df = df.dropna(subset=['Class'])  # Remover entradas com valores NaN na coluna 'Class'
X = df.drop(columns=['Class'])    # Variáveis preditoras
y = df['Class']                   # Variável alvo

## Validação Cruzada com Random Forest

Executa a validação cruzada k-fold para avaliar o desempenho do modelo Random Forest e exibe as acurácias obtidas em cada fold.

In [ ]:
# Executar k-fold cross-validation com 5 folds
model = RandomForestClassifier(n_estimators=100, random_state=42)
scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

# Resultados de cross-validation
print("Scores de cada fold:", scores)
print("Acurácia média:", np.mean(scores))

## Treinamento e Avaliação do Modelo

Divide os dados em conjuntos de treinamento e teste, treina o modelo com os dados de treinamento, faz previsões e avalia o desempenho no conjunto de teste usando a matriz de confusão e o relatório de classificação.

In [ ]:
# Dividir os dados em conjuntos de treino e teste para visualizações e avaliações adicionais
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Treinar o modelo nos dados de treino completos
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avaliar o modelo com o conjunto de teste
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

# Calcular a acurácia para o conjunto de teste
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia no conjunto de teste: {accuracy:.4f}")

## Visualização dos Resultados

Gera um gráfico de pizza para visualizar a distribuição das transações previstas como fraudulentas e não fraudulentas pelo modelo.

In [ ]:
# Gráfico de pizza com os resultados das transações previstas pelo modelo
predicted_counts = pd.Series(y_pred).value_counts().sort_index()
predicted_labels = ['Não Fraudulenta', 'Fraudulenta']
predicted_counts.plot(
    kind='pie',
    labels=[f"{label}: {count}" for label, count in zip(predicted_labels, predicted_counts)],
    startangle=140,
    colors=['skyblue', 'salmon'],
    autopct='%1.1f%%'
)
plt.title('Distribuição das Transações Previstas (Fraudulenta vs Não Fraudulenta)')
plt.ylabel('')
plt.show()